In [ ]:
import nest_asyncio

nest_asyncio.apply()

import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core import PromptTemplate

os.environ["OPENAI_API_KEY"] = ""

llm = OpenAI(model="gpt-4o-mini", temperature=0.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

documents = SimpleDirectoryReader("data").load_data()
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)
nodes = splitter.get_nodes_from_documents(documents)

df = pd.read_csv("questions/Lyft2021_queries.csv")
queries = df["Query"].tolist()

retriever = VectorStoreIndex(nodes).as_retriever(similarity_top_k=2)

In [ ]:
template = (
    "You are an intelligent assistant. Analyze the following context retrieved for the query:\n"
    "Query: {query}\n"
    "Context: {retrieved_context}\n"

    "Tasks:\n"
    "1. Summarize the most relevant information from the context.\n"
    "2. Identify any missing information or gaps in the context needed to fully answer the query.\n"
    "3. Suggest a refined query to improve retrieval if needed.\n"

    "Provide your response in the following format:\n"
    "- Refined Query: <summary>. <refined query>\n"
)

context_examination_template = PromptTemplate(template)

In [ ]:
def examine_context(query, retrieved_context, llm):
    """
    Prompts the LLM to examine the retrieved context and provide insights.

    Args:
        query (str): The current query.
        retrieved_context (str): Combined text of retrieved documents.
        llm: The language model instance.

    Returns:
        str: The response from the LLM.
    """
    # Use the prompt template to structure the input
    formatted_prompt = context_examination_template.format(
        query=query,
        retrieved_context=retrieved_context
    )

    # Pass the formatted prompt to the LLM
    response = llm.complete(formatted_prompt)
    return response

In [ ]:
def iterative_retrieval_with_context_examination(initial_query, retriever, llm, max_iterations=3):
    current_query = initial_query
    all_retrieved_docs = []
    refined_queries = []
    examination_results = []

    for iteration in range(max_iterations):
        # Retrieve documents
        retrieved_docs = retriever.retrieve(current_query)
        retrieved_info = [{"node_id": doc.node_id, "content": doc.get_content()} for doc in retrieved_docs]
        all_retrieved_docs.extend(retrieved_info)

        # Combine retrieved content for context
        combined_context = " ".join([doc.get_content() for doc in retrieved_docs])

        # Examine the context using the LLM
        examination_result = examine_context(current_query, combined_context, llm)
        examination_results.append(examination_result)

        # Parse the refined query from the examination result
        refined_query = examination_result.text.split("Refined Query:")[1].strip()
        refined_queries.append(refined_query)

        # Update the query for the next iteration
        if refined_query == current_query:
            print("Query stabilized; stopping iterations.")
            break
        current_query = refined_query

    return all_retrieved_docs, refined_queries, examination_results

In [ ]:
qa_dataset = []
for query in queries:
    retrieved_docs, refined_queries, examination_results = iterative_retrieval_with_context_examination(
        query, retriever, llm
    )

    qa_dataset.append({
        "query": refined_queries[-1],
        "retrieved_context": [doc["node_id"] for doc in retrieved_docs]
    })

In [ ]:
from llama_index.core.evaluation import RetrieverEvaluator

metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
evaluator = RetrieverEvaluator.from_metric_names(metrics, retriever=retriever)

eval_results = []
for entry in qa_dataset:
    query = entry["query"]
    context = entry["retrieved_context"]

    result = evaluator.evaluate(
        query=query,
        expected_ids=context
    )
    eval_results.append(result)

def display_results(name, eval_results):
    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [ ]:
display_results("iterative retrieval",  eval_results)